In [0]:
%load_ext autoreload
%matplotlib inline

In [0]:
#Import required libraries
%autoreload 2
!pip3 install --upgrade pip
!pip3 install --upgrade torch
!pip3 install torchvision
!pip3 install torchfusion
!pip3 install tensorboardx
!pip3 install pillow
!pip3 install pydicom
!pip3 install opencv-python
!pip3 install -q keras==2.2.4
!pip3 install --upgrade keras.utils
!pip3 install nibabel
!pip install graph_nets
#!pip3 install --upgrade tensorflow-gpu

import os
import errno
import scipy
import pydicom as dicom
import scipy.misc
import numpy as np
import tqdm
import nibabel as nib
#import cv2
import datetime
from IPython.display import clear_output

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow import nn, layers
from tensorflow.contrib import layers as clayers 

import keras
import keras.backend as K
from keras.layers.core import Dropout, Lambda, Dense, Flatten
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import Add, Concatenate
from keras.layers import Input, Activation, Flatten, ZeroPadding2D, UpSampling2D, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.engine import InputSpec
from keras.engine.topology import Layer
from keras.utils import conv_utils

import torch
import torch.cuda as cuda
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torch.distributions import Normal

import torchvision.utils as vutils
from torchvision import transforms, utils, datasets

from tensorboardX import SummaryWriter
from IPython import display

from PIL import Image

from getpass import getpass

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [0]:
class Logger:

    def __init__(self, model_name, data_name):
        
        self.model_name = model_name
        self.data_name = data_name

        self.comment = '{}_{}'.format(model_name, data_name)
        self.data_subdir = '{}/{}'.format(model_name, data_name)

        # TensorBoard
        self.writer = SummaryWriter(comment=self.comment)

    def log(self, error, epoch, n_batch, num_batches):

        # var_class = torch.autograd.variable.Variable
        if isinstance(error, torch.autograd.Variable):
            error = error.data.cpu().numpy()

        step = Logger._step(epoch, n_batch, num_batches)
        self.writer.add_scalar(self.comment + "/Error", error, step)
        

    def display_status(self, epoch, num_epochs, n_batch, num_batches, error):
        
        if isinstance(error, torch.autograd.Variable):
            error = error.data.cpu().numpy()
        
        clear_output()
        print('Epoch: [{}/{}], Batch Num: [{}/{}]'.format(
            epoch,num_epochs, n_batch, num_batches)
             )
        print('Loss: {:.4f}'.format(error))

    def save_models(self, model, epoch):
        out_dir = './data/models/{}'.format(self.data_subdir)
        Logger._make_dir(out_dir)
        torch.save(model.state_dict(),
                   '{}/epoch_{}'.format(out_dir, epoch))

    def close(self):
        self.writer.close()

    # Private Functionality

    @staticmethod
    def _step(epoch, n_batch, num_batches):
        return epoch * num_batches + n_batch

    @staticmethod
    def _make_dir(directory):
        try:
            os.makedirs(directory)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

In [0]:
class UCLADataset(Dataset):
          
    def __init__(self, transforms=None):
        
        self.transforms = transforms
        
        #get data from private BitBucket (since the data is not available publicly)
        os.chdir("/floyd/home/")
        user = getpass('BitBucket user')
        password = getpass('BitBucket password')
        os.environ['BITBUCKET_AUTH'] = user + ':' + password.replace("@", "%40")

        !git clone https://$BITBUCKET_AUTH@bitbucket.org/sidijju/ucla-data.git
        
        #Floydhub: 
        root = "/floyd/home/ucla-data/connectivity-data"

        for dirname, subdirname, filenames in os.walk(root):
            for filename in filenames:
                #read connectivity matrix in
                x = f.read().split(" ")
                arr = []
                count = 0
                for element in x:
                    if element != '':
                        arr.append(float(element))
                        count += 1
                size = int(math.sqrt(count))

                img = [[0] * size for i in range(size)]
                for i in range(size):
                    for j in range(size):
                        img[i][j] = arr[size*i + j]

                if filename == filenames[0]:
                    self.data = np.array(img)
                else:
                    self.data = np.concatenate((self.data, np.array(img)), axis = 2)
            print("Data Shape", self.data.shape)

        
    def __getitem__(self, index):
        if index < np.size(self.arr,0) and index >= 0:
            if self.transforms is not None:
                self.data.add(self.transforms(self.arr[index].astype('uint8')))
            return self.data[index].astype('uint8'), index
        else:
            print("INDEX INVALID")
            return None
        
    def __len__(self):
        return self.data.shape[0]
      

In [0]:
def ucla_data():
    DATA_FOLDER = './tf_data/GNN/Data'
    compose = transforms.Compose(
        [
            transforms.ToTensor()
        ])
    out_dir = '{}/dataset'.format(DATA_FOLDER)
    return UCLADataset(transforms = compose)

In [7]:
dataset = ucla_data()

FileNotFoundError: ignored

In [0]:
from keras.utils import plot_model
g = generator_model()
#plot_model(g, to_file='generator_name.png', show_shapes=True, show_layer_names=True)
g.load_weights("generator_sr50_" + str(currEpoch) + ".h5")
print("GENERATOR")
g.summary()

d = discriminator_model()
#plot_model(d, to_file='discriminator_name.png', show_shapes=True, show_layer_names=True)
d.load_weights("discriminator_sr50_" + str(currEpoch) + ".h5")
print("DISCRIMINATOR")
d.summary()

d_on_g = gendis_mult_out(g, d)

In [0]:
def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    os.chdir("/floyd/home/")
    save_dir = os.path.join("weights/", '{}_{}_{} run'.format(now.month, now.day, now.year))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_sr{}_{}.h5'.format(SAMPLING_RATE, epoch_number + currEpoch)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_sr{}_{}.h5'.format(SAMPLING_RATE, epoch_number + currEpoch)), True)

In [0]:
# Initialize optimizers
g_opt = Adam(epsilon=1E-4)
#lr - sr2 - 1E-4
d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# Compile models
d.trainable = True
d.compile(optimizer=d_opt, loss=d_loss)
d.trainable = False

loss = [g_loss, d_loss]
loss_weights = [100, 1]
d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
d.trainable = True

In [0]:
output_true_batch, output_false_batch = np.random.uniform(low = 0.9, high = 1.0, size = (BATCH_SIZE, 1)), np.zeros((BATCH_SIZE, 1))

logger = Logger(model_name='MRIGAN Trial 14.5 SR ' + str(SAMPLING_RATE) + '%', data_name='IXI')

In [0]:
x_train, y_train = get_data_simple_IXI(dataset, sampling_rate = int(100/SAMPLING_RATE))

In [0]:
for epoch in tqdm.tqdm(range(EPOCH_NUM)):
    
    num_samples = int(x_train.shape[2])
    
    permutated_indexes = np.random.permutation(num_samples)
    num_batches = int(num_samples / BATCH_SIZE)
    
    d_losses = []
    d_on_g_losses = []
    
    for n_batch in range(num_batches):
        batch_indexes = permutated_indexes[n_batch*BATCH_SIZE:(n_batch+1)*BATCH_SIZE]

        sampled_batch = K.eval(K.expand_dims(np.rollaxis(x_train[:, :, batch_indexes], 2, 0), axis = -1))
        full_batch = K.eval(K.expand_dims(np.rollaxis(y_train[:, :, batch_indexes], 2, 0), axis = -1))

        generated_images = g.predict(x=sampled_batch, batch_size=BATCH_SIZE)

        d_loss_real = d.train_on_batch(full_batch, output_true_batch)
        d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
        d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
        d_losses.append(d_loss)

        d.trainable = False

        d_on_g_loss = d_on_g.train_on_batch(sampled_batch, [full_batch, output_true_batch])
        d_on_g_losses.append(d_on_g_loss)

        d.trainable = True
        
        logger.log(d_loss, d_on_g_loss, epoch, n_batch, num_batches) 
        logger.log_images(generated_images, full_batch, sampled_batch, BATCH_SIZE, epoch, n_batch, num_batches, format='NHWC')   
    
    #logger.display_status(epoch, EPOCH_NUM, n_batch, num_batches, d_loss, d_on_g_loss, d_loss_real, d_loss_fake)
        
       
    #print(np.mean(d_losses), np.mean(d_on_g_losses))
    with open('log.txt', 'a+') as f:
        f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

    save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))